In [18]:
import glob
import os
from distutils.dir_util import copy_tree
import shutil
from pathlib import Path
from tqdm import tqdm

WORKDIR = "/work3/s202821/models/"
OUTDIR = "/work3/s202821/emp_models/"

In [19]:
runs_list = [
    "frcnn-hybrid_dwa-10v1-pcgrad_23-02-21T144644",
    "frcnn-hybrid_bmtl-graddrop_23-02-20T140735",
    "frcnn-hybrid_15v1_23-02-20T132858",
    "frcnn-hybrid_bmtl-cagrad_23-02-20T132609",
    "frcnn-hybrid_bmtl-pcgrad_23-02-20T132549",
    "frcnn-hybrid_10v1-graddrop_23-02-19T151850",
    "frcnn-hybrid_10v1-cagrad_23-02-19T151821",
    "frcnn-hybrid_10v1-pcgrad_23-02-19T151738",
    "frcnn-hybrid_dwa-cagrad_23-02-16T235038",
    "frcnn-hybrid_dwa-pcgrad_23-02-16T235026",
    "frcnn-hybrid_dwa_23-02-16T234921",
    "frcnn-hybrid_dwa-graddrop_23-02-16T234854",
    "frcnn-hybrid_uc-cagrad_23-02-16T140258",
    "frcnn-hybrid_uc-pcgrad_23-02-16T013822",
    "frcnn-hybrid_relative_23-02-16T004645",
    "frcnn-hybrid_bmtl_23-02-16T003206",
    "frcnn-hybrid_5v1_23-02-16T001810",
    "frcnn-hybrid_10v1_23-02-16T001236",
    "frcnn-hybrid_uc-graddrop_23-02-16T000240",
    "frcnn-hybrid_uncertainty_23-02-16T000024",
    "frcnn-hybrid_cagrad_23-02-15T235953",
    "frcnn-hybrid_graddrop_23-02-15T235924",
    "frcnn-hybrid_pcgrad_23-02-15T155512",
    "frcnn-hybrid_equal_23-02-14T235356",
    "ssdlite-hybrid_equal_23-02-14T235500",
    "lraspp-hybrid_equal_23-02-14T235345",
    "deeplabv3_23-02-14T235445",
    "ssdlite_23-02-14T235429",
    "frcnn_23-02-14T235417",
]

for run in runs_list:
    assert Path(WORKDIR + run).exists(), f"{run} does not exist"

In [20]:
for run in tqdm(runs_list):
   shutil.copytree(WORKDIR + run, OUTDIR + run)
    # copy_tree(WORKDIR + run, OUTDIR + run)

100%|██████████| 29/29 [00:19<00:00,  1.46it/s]
